In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [13]:
response.choices[0].message.content

"It depends on the specific course and the institution offering it. Many courses have a late enrollment period that allows students to join after the official start date, often with certain conditions or limitations. It's best to contact the course instructor or the admissions office of the institution offering the course to inquire about late enrollment options and any potential consequences, such as missed coursework or additional fees."

In [14]:
def search(query): 
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [17]:
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
rag(query)

"To run Kafka, if you are working with Java, you should execute the following command in your project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are working with Python, consider creating a virtual environment and running the requirements.txt in that environment. This ensures that all necessary dependencies are available for running your Kafka producer/consumer:\n\n```bash\n# Create a virtual environment (run only once)\npython -m venv env\n\n# Activate the virtual environment\nsource env/bin/activate\n\n# Install required packages\npip install -r ../requirements.txt\n```\n\nEnsure you activate the virtual environment every time you need to run the Python files:\n\n```bash\nsource env/bin/activate\n```\n\nDeactivate the virtual environment when you're done:\n\n```bash\ndeactivate\n```\n\nThis process works on MacOS and Linux. For Windows, the activation command is slightly different:\n\n```bash\n

In [19]:
rag('the course has already started, can i still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register right away, you are still eligible to submit the homework assignments. However, be mindful of the deadlines for submitting the final projects, so it’s best not to leave everything until the last minute."

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}